In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
def add_past_trials():
    list_of_past_trials = glob.glob('./../Past_Trials/csv_results/'+'*.csv')
    all_trials = pd.DataFrame()
    for trial in list_of_past_trials:
        print(trial)
        trial_df = pd.read_csv(trial)
        all_trials = pd.concat([all_trials,trial_df],axis=0)
    cleaned_all_trials = all_trials.loc[:, ["path_location", "Kernel", "BIC"]]
    cleaned_all_trials.to_csv("./../Past_Trials/cleaned_all_trials.csv", index=False)
    return cleaned_all_trials

In [3]:
# all_past_trials = pd.read_csv("./../Past_Trials/full_results/cleaned_all_trials.csv")
# all_past_trials.sort_values(by="BIC", inplace=True)
# list_of_kernels_ranked = all_past_trials.loc[:, ["Kernel"]]
# all_past_trials


In [7]:
trials_with_cv = pd.read_csv("top_10_trials_with_cv.csv")
# trials_with_cv.sort_values(by="BIC", inplace=True)
trials_with_cv.columns = ["BIC_rank", "Kernel", "BIC", "Error"]
trials_with_cv.sort_values(by="Error", inplace=True)
trials_with_cv

,BIC_rank,Kernel,BIC,Error
9,9,'AR2+RQ*AR2',-2326.191406,0.095826
8,8,'RBF+AR2*Per_Year*RBF*Mat_1.5+RBF',-2326.375732,0.095949
6,6,'RBF+AR2+Mat_2.5',-2333.447754,0.095949
5,5,'RBF+AR2*Per_Year*RBF*Mat_1.5+Per_Month',-2335.052002,0.096061
1,1,'RBF+AR2*Per_Year*RBF*Mat_1.5*RBF',-2350.953613,0.096677
2,2,'RBF+AR2*Per_Year*RBF',-2346.211914,0.096680
4,4,'RBF+AR2*Per_Year*RBF*Mat_1.5*Mat_2.5',-2335.255371,0.096698
0,0,'RBF+AR2*Per_Year*RBF*Mat_1.5',-2355.538086,0.096761
3,3,'RBF+AR2*Per_Year',-2337.717529,0.096776
7,7,'RBF+AR2*Per_Year*RBF+Per_Month',-2333.310303,0.096863
